<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-04-21 10:22:20
-------------------
qualified stocks: 90
with latest results: 84
still star stocks: 46
-------------------
Initial Investment:  1.09 C
CY Investment:  1.36 C
Reserve:  2.13 L
Current:  1.18 C
-------------------
Today PnL: 1.16 L (0.99%)
Current PnL: -18.59 L (-13.66%)
CY Booked + Current PnL: -18.30 L (-13.45%)
-------------------
Total profit:  5.04 L
Total loss:  -23.63 L
-------------------
Total Booked + Current PnL: 8.68 L (7.96%)
Total Booked PnL: 27.27 L (25.01%)
Curr Year Booked PnL: 28.87 K (0.25%)
Prev Year Booked PnL: 26.98 L (24.74%)
Est FTT:  1.93 C
Est FTT PnL: 75.52 L (64.11%)
-------------------
Est LTT:  2.54 C
Est LTT PnL: 1.36 C (115.23%)
Deployed:  1.09 C
Current:  1.18 C
CAGR/XIRR %: 5.55%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SBIN,760.30,863.00,-16.89,M-LC,78.89,217207.0,15728.0,11490.0,2.78,7.81,5.29,13.51,93.0,1.37,1.88,19.00,XY25,NTT,BANKS
11,AXISBANK,1003.18,1317.30,-11.26,H-LC,92.22,149023.0,26635.0,11683.0,2.58,21.76,7.84,31.31,NaN,2.28,1.29,28.84,X40,ATH,BANKS
54,NMDC,65.62,82.00,37.25,M-MC,25.56,102280.0,1881.0,23177.0,1.49,1.87,22.66,24.96,507.0,0.08,0.88,15.28,XY25,NTT,MINING
69,SBILIFE,1496.49,1924.99,-8.78,H-LC,48.89,135332.0,9627.0,26363.0,0.18,7.66,19.48,28.63,NaN,0.37,1.17,20.56,AR,ATH,INSURANCE
85,UJJIVANSFB,59.50,60.00,0.11,M-SC,98.89,74834.0,-26316.0,27165.0,4.56,-26.02,36.30,0.84,-27.0,-0.97,0.65,40.28,X40N,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.0,-277.02,H-SC,100.00,96005.0,-37281.0,121878.0,8.63,-27.97,126.95,63.47,-27.0,-0.31,0.83,60.93,XR,NTT,CHEMICALS
79,TATAELXSI,7332.28,9000.0,-33.46,X-MC,15.56,72058.0,-30594.0,53943.0,5.03,-29.80,74.86,22.74,-10.0,-0.57,0.62,8.31,X40N,NTT,IT
57,QUESS,711.66,986.0,-62.82,M-SC,1.11,75577.0,-80277.0,140354.0,4.99,-51.51,185.71,38.55,-33.0,-0.57,0.65,15.73,XY24,NTT,MISC
85,UJJIVANSFB,59.50,60.0,0.11,M-SC,98.89,74834.0,-26316.0,27165.0,4.56,-26.02,36.30,0.84,-27.0,-0.97,0.65,40.28,X40N,NTT,BANKS
0,5PAISA,564.75,810.0,-48.32,H-SC,87.78,131308.0,-55624.0,136797.0,4.23,-29.76,104.18,43.43,20.0,-0.41,1.14,25.62,X40N,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ADANIPORTS,1103.69,1583.00,-4.40,M-LC,45.56,168565.0,17359.0,48311.0,-2.30,11.48,28.66,43.43,NaN,0.36,1.46,16.84,XY24,NTT,MISC
21,COFFEEDAY,59.14,80.00,-125.35,L-SC,24.44,53626.0,-59923.0,99975.0,-2.00,-52.77,186.43,35.27,NaN,-0.60,0.46,31.25,XR,NTT,HOTELS
34,HDFCLIFE,560.54,897.00,-3.60,H-LC,50.00,186559.0,39137.0,49345.0,-1.49,26.55,26.45,60.02,16.0,0.79,1.61,33.18,X40,BTT,INSURANCE
28,GICRE,335.06,583.00,4.43,H-MC,16.67,185572.0,39151.0,69200.0,-1.06,26.74,37.29,74.00,109.0,0.57,1.60,35.71,XY24,BTT,INSURANCE
5,ALKYLAMINE,2347.98,4567.52,-9.90,X-SC,94.44,80453.0,-20510.0,115949.0,-1.06,-20.31,144.12,94.53,-59.0,-0.18,0.70,19.28,SR,ATH,CHEMICALS


In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,NHPC,82.08,115.00,217.46,M-MC,52.22,148607.0,7594.0,48966.0,1.79,5.38,32.95,40.11,-54.0,0.16,1.28,21.49,XY24,NTT,POWER
54,NMDC,65.62,82.00,37.25,M-MC,25.56,102280.0,1881.0,23177.0,1.49,1.87,22.66,24.96,507.0,0.08,0.88,15.28,XY25,NTT,MINING
28,GICRE,335.06,583.00,4.43,H-MC,16.67,185572.0,39151.0,69200.0,-1.06,26.74,37.29,74.00,109.0,0.57,1.60,35.71,XY24,BTT,INSURANCE
19,CAMS,3643.00,5250.99,-0.66,H-SC,44.44,110370.0,8366.0,36654.0,1.53,8.20,33.21,44.14,261.0,0.23,0.95,29.25,XR,ATH,MISC
67,SBICARD,765.91,1075.00,-2.02,H-MC,65.56,215554.0,35565.0,37075.0,1.20,19.76,17.20,40.36,-1.0,0.96,1.86,38.58,XY24,NTT,FINANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMS,3643.00,5250.99,-0.66,H-SC,44.44,110370.0,8366.0,36654.0,1.53,8.20,33.21,44.14,261.0,0.23,0.95,29.25,XR,ATH,MISC
13,BANKINDIA,112.90,190.00,-24.62,M-MC,86.67,105619.0,5364.0,63097.0,1.57,5.35,59.74,68.29,-12.0,0.09,0.91,31.64,XR,NTT,BANKS
1,AARTIIND,487.04,919.00,10.11,M-SC,31.11,68328.0,-13008.0,85144.0,2.35,-15.99,124.61,88.69,-8.0,-0.15,0.59,15.22,XR,NTT,CHEMICALS
75,SONACOMS,578.05,1006.00,-39.28,M-MC,6.67,79502.0,-21657.0,96547.0,0.57,-21.41,121.44,74.03,-8.0,-0.22,0.69,11.73,AR,BTT,AUTO
65,SAMMAANCAP,170.35,326.00,-161.84,M-SC,46.67,73374.0,-28836.0,122226.0,1.70,-28.21,166.58,91.37,NaN,-0.24,0.63,19.11,XR,NTT,FINANCE


In [14]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,59.50,60.0,0.11,M-SC,98.89,74834.0,-26316.0,27165.0,4.56,-26.02,36.30,0.84,-27.0,-0.97,0.65,40.28,X40N,NTT,BANKS
88,VALIANTORG,512.64,838.0,-277.02,H-SC,100.00,96005.0,-37281.0,121878.0,8.63,-27.97,126.95,63.47,-27.0,-0.31,0.83,60.93,XR,NTT,CHEMICALS
13,BANKINDIA,112.90,190.0,-24.62,M-MC,86.67,105619.0,5364.0,63097.0,1.57,5.35,59.74,68.29,-12.0,0.09,0.91,31.64,XR,NTT,BANKS
86,UNIONBANK,123.87,163.0,-8.95,M-LC,56.67,147003.0,6163.0,38324.0,2.13,4.38,26.07,31.59,-39.0,0.16,1.27,27.69,XY24,NTT,BANKS
11,AXISBANK,1003.18,1317.3,-11.26,H-LC,92.22,149023.0,26635.0,11683.0,2.58,21.76,7.84,31.31,NaN,2.28,1.29,28.84,X40,ATH,BANKS


In [15]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.0,-277.02,H-SC,100.00,96005.0,-37281.0,121878.0,8.63,-27.97,126.95,63.47,-27.0,-0.31,0.83,60.93,XR,NTT,CHEMICALS
85,UJJIVANSFB,59.50,60.0,0.11,M-SC,98.89,74834.0,-26316.0,27165.0,4.56,-26.02,36.30,0.84,-27.0,-0.97,0.65,40.28,X40N,NTT,BANKS
77,SURYODAY,189.78,240.0,-36.78,H-SC,97.78,103332.0,-46594.0,86272.0,2.77,-31.08,83.49,26.46,-41.0,-0.54,0.89,32.38,XR,NTT,BANKS
38,INDUSINDBK,1354.50,1800.0,-36.38,H-MC,96.67,47051.0,-30156.0,55548.0,3.87,-39.06,118.06,32.89,-21.0,-0.54,0.41,29.57,XR,NTT,BANKS
62,REPCOHOME,515.07,880.0,-12.33,H-SC,95.56,161568.0,-43430.0,188679.0,1.92,-21.19,116.78,70.85,-44.0,-0.23,1.40,30.43,XY24,NTT,FINANCE


In [16]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,42.49
2,50,72.33


In [17]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    40.51
X40     13.60
X40N    12.86
AR      12.01
XR       9.35
XY25     9.26
SR       1.28
X200     1.10
Name: CurrAlloc%, dtype: float64

In [18]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-LC    20.61
H-SC    19.24
M-SC    17.48
M-MC     7.85
H-LC     7.74
M-LC     6.86
H-MC     6.68
X-MC     6.48
X-SC     2.78
L-SC     2.42
Name: CurrAlloc%, dtype: float64

In [19]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
BANKS,14.59,-4.14,52.79
FMCG,10.83,0.79,30.03
FINANCE,9.64,-15.34,66.62
INSURANCE,6.77,4.92,39.15
IT,6.74,-42.58,113.63
MISC,6.54,-36.31,85.63
PAINTS,6.12,-16.40,50.85
CHEMICALS,4.27,-9.46,81.58
ELECTRICAL,3.93,-18.66,70.88


In [20]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3173876.0,30
X40N,1188183.0,16
XR,1151667.0,14
AR,1014622.0,11
X40,571456.0,10
SR,197305.0,2
XY25,164453.0,6
X200,90786.0,1


In [21]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2139005.0,18
M-SC,2073848.0,21
X-LC,845200.0,14
M-MC,635376.0,7
X-SC,354979.0,4
L-SC,316308.0,4
H-MC,302789.0,5
X-MC,269229.0,5
M-LC,263337.0,5


In [22]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
M-SC       XY24      1015789.0      8
H-SC       XY24       928472.0      6
           X40N       534038.0      6
M-SC       XR         453897.0      5
M-MC       XY24       452555.0      4
H-SC       AR         394472.0      2
X-LC       X40        375813.0      6
M-SC       X40N       355009.0      5
H-SC       XR         282023.0      4
M-LC       XY24       251847.0      4
H-MC       XY24       247241.0      4
X-LC       AR         203832.0      3
L-SC       XR         169853.0      2
X-SC       XR         127249.0      1
X-MC       X40N       126092.0      2
X-SC       SR         115949.0      1
           X40N       111781.0      2
H-LC       AR         110150.0      2
X-LC       XY25       101236.0      3
M-MC       AR          96547.0      1
M-SC       AR          95524.0      1
X-LC       X200        90786.0      1
M-SC       SR          81356.0      1
L-SC       XY24        79972.0      1
X-LC       XY24        73533.0      1
M-SC       X40         72273.0      1
L-SC       AR          66483.0      1
M-MC       XR          63097.0      1
X-MC       X40         62342.0      1
H-LC       X40N        61263.0      1
           X40         61028.0      2
H-MC       XR          55548.0      1
X-MC       AR          47614.0      1
           XY24        33181.0      1
H-LC       XY25        28550.0      1
M-MC       XY25        23177.0      1
M-LC       XY25        11490.0      1

In [23]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 22.0 seconds
